## PROTEUS2 protein secondary structure analysis:

### This is a notebook to organize and evaluate the output of protein secondary structure prediction by [Proteus2](http://www.proteus2.ca/proteus2/), described in [Montgomerie et al., 2008](https://academic.oup.com/nar/article/36/suppl_2/W202/2506231) in _Nucleic Acids Research_. 

### The output of Proteus2 comes in an email in nominally FASTA format, but with extra line breaks and spacings within protein and prediction sequences. Also, the name gets cut off.

#### They look like this:

#### >Thalas

MMKLAALAAL MGSAAAFAPA QTGKASTQLR AFEDELGAQP PLGFFDPFGM 

CCHHHHHHHH HHHHHCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

LSGDCTQERF DRLRYVEIKH GRICMLAFLG QIVTRAGIHL PGSINYAGDS 

CCCCCCHHHH HHHHHHHHHH HHHHHHHHHH HHHHHHHCCC CCCCCCCCCC 

FDSFPNGVAA LFGPNSIPTA GLVQIIAFIG VLECAFMRDV PGTGNEFVGD 

CCCCCCCCCC CCCCCCCCHH HHHHHHHHHH HHHHHHHHCC CCCCCCCCCC 

FRNGYIDFGW DDFDEETKLQ KRAIQSGTIS NMMKLAALAA LMGSAAAFAP 

CCCCCCCCCC CCCCHHHHHH HHHHHHHHHH HHHHHHHHHC CCCCCCCCCC 



### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/

In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls

PeaksDB_322_prot_trypsin_totals
PeaksDB_323_prot_trypsin_totals
T0_322_trypsin_PeaksDB_protein_proteus
T0_322_trypsin_PeaksDB_protein_proteus_sort.csv
T0_322_trypsin_PeaksDB_protein_proteus.txt
T0_329_undigested_PeaksDB_protein_proteus
T12_325_trypsin_PeaksDB_protein_proteus
T2_323_trypsin_PeaksDB_protein_proteus
T2_323_trypsin_PeaksDB_protein_proteus_sort.csv
T2_323_trypsin_PeaksDB_protein_proteus.txt
T5_324_trypsin_PeaksDB_protein_proteus


In [4]:
!head T5_324_trypsin_PeaksDB_protein_proteus

>Thalas

MMKLAALAAL MGSAAAFAPA QTGKAFTQLR AFEDELGAQP PLGFFDPFGM 
CCCHHHHHHH HCCCCCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

LSGDCTQERF DRLRYVEIKH GRICMLAFLG QIVTRAGIHL PGSINYAGDS 
CCCCCCHHHH HHHHHHHHHH HHHHHHHHHH HHHHHHHHCC CCCCCCCCCC 

FDSFPNGVAA LFGPNSIPTA GLVQIIAFIG VLECAFMRDV PGTGNEFVGD 
HHHHCCCCCC CCCCCHHHHH HHHHHHHHHH HHHHHHHHHH CCCCCCCCCC 


In [5]:
# remove empty lines
# remove empty spaces from lines
# remove carat protein sequence names
# collapse the 2 lines from every protein sequence line (protein seq, secondard stucture pred.) into one line

!sed '/^[[:space:]]*$/d' T5_324_trypsin_PeaksDB_protein_proteus | cat \
| sed '/>/d' \
| tr -d "[:blank:]" > T5_324_trypsin_PeaksDB_protein_proteus.txt

!awk '{printf "%s%s",$0,(NR%2?FS:RS)}' T5_324_trypsin_PeaksDB_protein_proteus.txt > \
T5_324_trypsin_PeaksDB_protein_proteus_sort.csv

In [6]:
!head T5_324_trypsin_PeaksDB_protein_proteus_sort.csv

MMKLAALAALMGSAAAFAPAQTGKAFTQLRAFEDELGAQPPLGFFDPFGM CCCHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
LSGDCTQERFDRLRYVEIKHGRICMLAFLGQIVTRAGIHLPGSINYAGDS CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCC
FDSFPNGVAALFGPNSIPTAGLVQIIAFIGVLECAFMRDVPGTGNEFVGD HHHHCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCC
FRNGYIDFGWDDFNEETKLQKRAIELNNGRRNGYIDFGWDDFDEETKLQK CCCCCCCCCCCCCCCCHHHHCCCCCCCCCCCCCCCCCCCCCCCHHHHHHH
RAIELNNGRAAMMGILGLMVHEEIIPLGYDPDLPIIGHLQ HHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCHHHHCC
MMKLAALAALMGSAAAFAPAQTGKASTQLRAFEDELGAQPPLGFFDPFGM CCHHHHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
LSGDCTQERFDRLRYVEIKHGRICMLAFLGQIVTRAGIHLPGSINYAGDS CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCCC
FDSFPNGVAALFGPNSIPTAGLVQIIAFIGVLECAFMRDVPGTGNEFVGD CCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCC
FRNGYIDFGWDDFDEETKLQKRAIQSGTISNMMKLAALAALMGSAAAFAP CCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCC
AQTGKASTQLRAFEDELGAQPPLGFFDPFGMLSGDCTQERFDRLRYVEIK CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHH


In [7]:
# read into pandas with space delimeter

PeaksDB_324_prot = pd.read_csv("T5_324_trypsin_PeaksDB_protein_proteus_sort.csv", delim_whitespace=True, header=None)


# name columns

PeaksDB_324_prot.columns =['Stripped protein sequence', 'Secondary structure pred.'] 

In [8]:
PeaksDB_324_prot.head()

,Stripped protein sequence,Secondary structure pred.
0,MMKLAALAALMGSAAAFAPAQTGKAFTQLRAFEDELGAQPPLGFFD...,CCCHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
1,LSGDCTQERFDRLRYVEIKHGRICMLAFLGQIVTRAGIHLPGSINY...,CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCC...
2,FDSFPNGVAALFGPNSIPTAGLVQIIAFIGVLECAFMRDVPGTGNE...,HHHHCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCC...
3,FRNGYIDFGWDDFNEETKLQKRAIELNNGRRNGYIDFGWDDFDEET...,CCCCCCCCCCCCCCCCHHHHCCCCCCCCCCCCCCCCCCCCCCCHHH...
4,RAIELNNGRAAMMGILGLMVHEEIIPLGYDPDLPIIGHLQ,HHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCHHHHCC


In [9]:
# add a column with the stripped peptide length (number of AAs)
PeaksDB_324_prot['Sequence length'] = PeaksDB_324_prot['Stripped protein sequence'].apply(len)

# use a count function to enumerate the # of C's (coil residues) for each peptide
PeaksDB_324_prot['C'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("C")

# use a count function to enumerate the # of H's (helices residues) in each peptide
PeaksDB_324_prot['H'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("H")

# use a count function to enumerate the # of E's (beta strand residues) in each peptide
PeaksDB_324_prot['E'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("E")

#use a count function to enumerate the # of T's (membrane helix residues) in each peptide
PeaksDB_324_prot['T'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("T")

# use a count function to enumerate the # of B's (membrane strand residues) in each peptide
PeaksDB_324_prot['B'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("B")

# use a count function to enumerate the # of B's (signal peptide residues) in each peptide
PeaksDB_324_prot['S'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("S")

# use a count function to enumerate the # of c's (cleavage site residues) in each peptide
PeaksDB_324_prot['c'] = PeaksDB_324_prot['Secondary structure pred.'].str.count("c")

# add a column with the % C
PeaksDB_324_prot['% C'] = PeaksDB_324_prot['C'] / PeaksDB_324_prot['Sequence length']

# add a column with the % H
PeaksDB_324_prot['% H'] = PeaksDB_324_prot['H'] / PeaksDB_324_prot['Sequence length']

# add a column with the % E
PeaksDB_324_prot['% E'] = PeaksDB_324_prot['E'] / PeaksDB_324_prot['Sequence length']

# add a column with the % T
PeaksDB_324_prot['% T'] = PeaksDB_324_prot['T'] / PeaksDB_324_prot['Sequence length']

# add a column with the % B
PeaksDB_324_prot['% B'] = PeaksDB_324_prot['B'] / PeaksDB_324_prot['Sequence length']

# add a column with the % S
PeaksDB_324_prot['% S'] = PeaksDB_324_prot['S'] / PeaksDB_324_prot['Sequence length']

# add a column with the % c
PeaksDB_324_prot['% c'] = PeaksDB_324_prot['c'] / PeaksDB_324_prot['Sequence length']

# additive check

PeaksDB_324_prot['% check'] = PeaksDB_324_prot['% C'] + PeaksDB_324_prot['% H'] + PeaksDB_324_prot['% E'] \
                                + PeaksDB_324_prot['% T'] + PeaksDB_324_prot['% B'] + \
                                PeaksDB_324_prot['% S'] + PeaksDB_324_prot['% c']

In [10]:
PeaksDB_324_prot.head()

,Stripped protein sequence,Secondary structure pred.,Sequence length,C,H,E,T,B,S,c,% C,% H,% E,% T,% B,% S,% c,% check
0,MMKLAALAALMGSAAAFAPAQTGKAFTQLRAFEDELGAQPPLGFFD...,CCCHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,50,42,8,0,0,0,0,0,0.84,0.16,0.0,0.0,0.0,0.0,0.0,1.0
1,LSGDCTQERFDRLRYVEIKHGRICMLAFLGQIVTRAGIHLPGSINY...,CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCC...,50,18,32,0,0,0,0,0,0.36,0.64,0.0,0.0,0.0,0.0,0.0,1.0
2,FDSFPNGVAALFGPNSIPTAGLVQIIAFIGVLECAFMRDVPGTGNE...,HHHHCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCC...,50,21,29,0,0,0,0,0,0.42,0.58,0.0,0.0,0.0,0.0,0.0,1.0
3,FRNGYIDFGWDDFNEETKLQKRAIELNNGRRNGYIDFGWDDFDEET...,CCCCCCCCCCCCCCCCHHHHCCCCCCCCCCCCCCCCCCCCCCCHHH...,50,39,11,0,0,0,0,0,0.78,0.22,0.0,0.0,0.0,0.0,0.0,1.0
4,RAIELNNGRAAMMGILGLMVHEEIIPLGYDPDLPIIGHLQ,HHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCHHHHCC,40,12,28,0,0,0,0,0,0.30,0.70,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
index = ['324 total']

data = {
        '% C total': PeaksDB_324_prot['% C'].sum(),
        '% H total': PeaksDB_324_prot['% H'].sum(),
        '% E total': PeaksDB_324_prot['% E'].sum(),
        '% T total': PeaksDB_324_prot['% T'].sum(),
        '% B total': PeaksDB_324_prot['% B'].sum(),
        '% S total': PeaksDB_324_prot['% S'].sum(),
        '% c total': PeaksDB_324_prot['% c'].sum(),
        '% check sum': PeaksDB_324_prot['% check'].sum()
       }

PeaksDB_324_prot_totals = pd.DataFrame(data, columns=['% C total', '% H total', '% E total', '% T total', \
                                                      '% B total',  '% S total', '% c total', \
                                                      '% check sum'], index=index)

PeaksDB_324_prot_totals['overall % sum'] = PeaksDB_324_prot_totals['% C total'] \
                                            + PeaksDB_324_prot_totals['% H total'] \
                                            + PeaksDB_324_prot_totals['% E total'] \
                                            + PeaksDB_324_prot_totals['% T total'] \
                                            + PeaksDB_324_prot_totals['% B total'] \
                                            + PeaksDB_324_prot_totals['% S total'] \
                                            + PeaksDB_324_prot_totals['% c total'] 


PeaksDB_324_prot_totals['overall % C'] = PeaksDB_324_prot_totals['% C total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % H'] = PeaksDB_324_prot_totals['% H total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % E'] = PeaksDB_324_prot_totals['% E total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % T'] = PeaksDB_324_prot_totals['% T total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % B'] = PeaksDB_324_prot_totals['% B total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % S'] = PeaksDB_324_prot_totals['% S total'] / PeaksDB_324_prot_totals['overall % sum']

PeaksDB_324_prot_totals['overall % c'] = PeaksDB_324_prot_totals['% c total'] / PeaksDB_324_prot_totals['overall % sum']

# write to csv

PeaksDB_324_prot_totals.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/PeaksDB_324_prot_trypsin_totals")

PeaksDB_324_prot_totals.head()

,% C total,% H total,% E total,% T total,% B total,% S total,% c total,% check sum,overall % sum,overall % C,overall % H,overall % E,overall % T,overall % B,overall % S,overall % c
324 total,1131.70934,497.631236,248.278367,233.637899,11.743158,0.0,0.0,2123.0,2123.0,0.533071,0.2344,0.116947,0.110051,0.005531,0.0,0.0
